In [ ]:
import pandas as pd
import zipfile
import os
import time
from google.colab import drive
from PIL import Image

In [ ]:
!pip install pysrt
import pysrt

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# чтобы разгрузить код я буду путь к файлам сохранять в эту переменную
PATH = None

In [135]:
# создаем датафрейм
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/movies_labels.xlsx'
data = pd.read_excel(PATH)
data.head()

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [136]:
data.shape[0]

241

In [137]:
# читаем файл с субтитрами
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/Subtitles/10_Cloverfield_lane(2016).srt'
subs = pysrt.open(PATH)


In [138]:
#help (pysrt)

In [139]:
# я посмотрела как будут выглядеть субтитры
subtitle_texts = [subtitle.text for subtitle in subs]
#subtitle_texts

Для того, чтобы получить субтитры в удобноваримом формате, мы пройдемся циклом по всему содержимому папки Subtitles, которая содержит папки с названиями фильмов. Для этого используем модуль os, который позволяет работать с файлами и директориями.

In [140]:
# создадим пустые списки в которые соберем название и субтитры каждого фильма
subtitles = []
movie = []

Я выбирала между сохранить в словарь название:субтитры или два списка.
Решила сделать два списка, потому что потом мне нужно будет сравнивать фильмы в дф и в списке фильмов с субтитрами, а это легче сделать из списка, а потом добавить в дф.

In [141]:
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/Subtitles'

In [142]:
# сохраним названия файлов в папке Subtitles в список
film_list = os.listdir(PATH)
film_list[:5]

['Oceans_Twelve(2004).srt',
 'Oceans_Eleven(2001).srt',
 'Mona_Lisa_Smile(2003).srt',
 'Bridget_Joness_Baby.srt',
 'BrenВ.Brown.The.Call.to.Courage.2019.720.NF.720p.DDP.5.1.x264-CafeFlix.srt']

Обращаем внимание, что имеются строки в которых точка используется как разделитель, это важно, так как нам нужно сохранить названия фильмов без расширения .srt

---

Делить будем так:
- находим индекс где стоит **последняя** точка
- добавляем строку до этого индекса

In [143]:
# проверяем
last_dot = film_list[2].rfind('.')
film_list[2][:last_dot]

'Mona_Lisa_Smile(2003)'

In [144]:
len(os.listdir(PATH))

115

In [145]:
# создаем цикл, который будет проходить по элементам списка и сохранять название для кажлого фильма
for film in film_list:
    last_dot = film.rfind('.')
    movie.append(film[:last_dot])
movie[:10]

['Oceans_Twelve(2004)',
 'Oceans_Eleven(2001)',
 'Mona_Lisa_Smile(2003)',
 'Bridget_Joness_Baby',
 'BrenВ.Brown.The.Call.to.Courage.2019.720.NF.720p.DDP.5.1.x264-CafeFlix',
 'Zootopia(2016)',
 'Klaus(2019)',
 'The_Legend_of_Tarzan(2016)',
 'Despicable_Me(2010)',
 'Made_of_Honor(2008)']

В работе цикла возникла ошибка что файл не найден в указанной директории.
Я попробовала открыть отдельно этот файл(вручную прописав путь к нему) и возникла ошибка кодировки. Чтобы решить эти вопросы:
- Поэтому добавила в цикл try-except, чтобы обработать все возможные кодировки
- Создала переменную путь и название, чтобы избежать того, что возникает ошибка из-за некорректной обработки названий.


In [146]:
# создаем цикл, который будет проходить по элементам списка и сохранять субтитры
for film in film_list:
  file_to_read = os.path.join(PATH, film)
  try:
    encoded_film = pysrt.open(file_to_read, encoding='iso-8859-1')
  except:
    encoded_film = pysrt.open(file_to_read, encoding='UTF-8')

  subs_text = [film.text for film in encoded_film]
  subtitles.append(subs_text)


In [147]:
len(subtitles)

115

In [148]:
len(movie)

115

In [149]:
movie.count('None')

0

Теперь можно обработать названия фильмов, чтобы добавить субтитры к фильмам.
Обрабатывать буду следующими шагами:
- все слова к строчному регистру
- в качестве разделителя только знак нижнего подчеркивания "_"
- удалю все пробелы, знаки между словами, оставлю только цифры.

In [150]:
data.head()

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [151]:
def transform_values(df):
    # заменяем пробелы на underscore
    df = df.applymap(lambda x: x.replace(' ', '_') if isinstance(x, str) else x)

    # заменяем дефис на underscore
    df = df.applymap(lambda x: x.replace('-', '_') if isinstance(x, str) else x)

    # добавляем underscore между словом и цислом
    df = df.applymap(lambda x: '_'.join(part for part in x.split() if not part.isdigit()) if isinstance(x, str) else x)

    # приводим к строчному регистру
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    return df


In [152]:
# почистим названия фильмов в дата
data = transform_values(data)
data.head()

,id,Movie,Level
0,0,10_cloverfield_lane(2016),b1
1,1,10_things_i_hate_about_you(1999),b1
2,2,a_knights_tale(2001),b2
3,3,a_star_is_born(2018),b2
4,4,aladdin(1992),a2/a2+


In [153]:
data.columns

Index(['id', 'Movie', 'Level'], dtype='object')

In [154]:
subs_df = pd.DataFrame({'subs': subtitles, 'Movie' : movie})
subs_df.head()

,subs,Movie
0,"[Hi., - How'd it go?\n- Lousy., Got a '63 Thun...",Oceans_Twelve(2004)
1,[<i>GUARD: One con under escort.\nOpen gate on...,Oceans_Eleven(2001)
2,[All her life she had wanted\nto teach at Well...,Mona_Lisa_Smile(2003)
3,"[Happy birthday to me\nHappy birthday to me, H...",Bridget_Joness_Baby
4,[[presenter]\n<i>She spent 20 years studying c...,BrenВ.Brown.The.Call.to.Courage.2019.720.NF.72...


In [155]:
subs_df['Movie'] = transform_values(subs_df[['Movie']])
subs_df['Movie'].head()

0                                  oceans_twelve(2004)
1                                  oceans_eleven(2001)
2                                mona_lisa_smile(2003)
3                                  bridget_joness_baby
4    brenв.brown.the.call.to.courage.2019.720.nf.72...
Name: Movie, dtype: object

In [156]:
subs_df.head()

,subs,Movie
0,"[Hi., - How'd it go?\n- Lousy., Got a '63 Thun...",oceans_twelve(2004)
1,[<i>GUARD: One con under escort.\nOpen gate on...,oceans_eleven(2001)
2,[All her life she had wanted\nto teach at Well...,mona_lisa_smile(2003)
3,"[Happy birthday to me\nHappy birthday to me, H...",bridget_joness_baby
4,[[presenter]\n<i>She spent 20 years studying c...,brenв.brown.the.call.to.courage.2019.720.nf.72...


Сделаю тип объединения outer так как могут удалиться дубликаты, а я хочу сохранить все наблюдения и отдельно обработать дубликаты чтобы избежать ошиб

In [157]:
data = pd.merge (data, subs_df,
                 on = 'Movie',
                 how = 'outer',
                 indicator = True)
data.head()

,id,Movie,Level,subs,_merge
0,0,10_cloverfield_lane(2016),b1,"[<font color=""#ffff80""><b>Fixed & Synced by bo...",both
1,1,10_things_i_hate_about_you(1999),b1,"[Hey!, I'll be right with you., So, Cameron. H...",both
2,2,a_knights_tale(2001),b2,"[Resync: Xenzai[NEF]\nRETAIL, Should we help h...",both
3,3,a_star_is_born(2018),b2,"[- <i><font color=""#ffffff""> Synced and correc...",both
4,4,aladdin(1992),a2/a2+,"[<i>Oh, I come from a land\nFrom a faraway pla...",both


In [158]:
data['_merge'].value_counts()

left_only     131
both          110
right_only      0
Name: _merge, dtype: int64

In [160]:
data.shape[0]

241

In [ ]:
!git init

In [ ]:
!git clone https://github.com/AV-BOLT/Masterskaya_2.git

fatal: destination path 'Masterskaya_2' already exists and is not an empty directory.


In [ ]:
!git remote add origin https://github.com/AV-BOLT/Masterskaya_2.git

In [ ]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	English_scores.zip
	English_scores.zip (Unzipped Files)/
	Masterskaya_2/
	Notebooks/

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
ls


 English_scores.zip                      Masterskaya_2/
'English_scores.zip (Unzipped Files)'/   Notebooks/


In [ ]:
!git commit -m 'Создала файл ноутбука'

[master 8fada91] Создала файл ноутбука
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
!git config --global user.email "gelika176@.com"
!git config --global user.name "Anzhelika Boltneva"

In [ ]:
!git add .

In [ ]:
!git commit -m 'Создала новый репозиторий и перезалила ноутбук'

[master (root-commit) 3c7ba11] Создала новый репозиторий и перезалила ноутбук
 1 file changed, 1 insertion(+)
 create mode 100644 determine_langlevel_by_subs.ipynb


In [ ]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   determine_langlevel_by_subs.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add determine_langlevel_by_subs.ipynb

In [ ]:
!git status

:wq
On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   determine_langlevel_by_subs.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   determine_langlevel_by_subs.ipynb



In [ ]:
!pwd

'/content/drive/MyDrive/Masterskaya_2/Notebooks'

# Новый раздел